In [10]:
date='0704'

WIN PROBABILITY DATA

In [ ]:
import pandas as pd
import requests
from audl.stats.endpoints.gameevents import *
import numpy as np

features = ['thrower_x', 'thrower_y', 'start_on_offense', 'point_start_time', 'possession_num', 'possession_throw', 'game_quarter', 'quarter_point', 'is_home_team', 'home_team_score', 'away_team_score', 'gameID', 'total_points', 'home_teamID', 'away_teamID']

def process_games(GAMES):
    GAMES['total_points'] = GAMES['home_team_score'] + GAMES['away_team_score']
    GAMES = GAMES[GAMES.game_quarter < 5] ##TODO include overtimes
    GAMES['point_start_time'] = 720 - GAMES['point_start_time']
    GAMES = GAMES[features]

    games = []
    for gameID in GAMES.gameID.unique():
        GAME = GAMES[GAMES.gameID == gameID]
        points = []
        prev_point_time = 720
        current_quarter = 1
        prev_df = None
        error_points = 0
        for group_keys, group_df in GAME.groupby(['total_points', 'game_quarter']):
            if prev_df is None:
                prev_df = group_df
                continue

            prev_point_time = prev_df.point_start_time.max()
            if (current_quarter != group_df.game_quarter.max()):
                current_quarter = group_df.game_quarter.max()
                current_point_time = 0
            else:
                current_point_time = group_df.point_start_time.max()
            if current_point_time >= prev_point_time:
                error_points = error_points + 1
                continue
            times = np.linspace(prev_point_time, current_point_time + 1, len(prev_df))
            prev_df['times'] = times
            points.append(prev_df)
            prev_df = group_df
            if group_keys == (GAME.home_team_score.max()+GAME.away_team_score.max(), 4):
                times = np.linspace(current_point_time, 0, len(prev_df))
                prev_df['times'] = times
                points.append(prev_df)
        GAME_POINTS = pd.concat(points)
        GAME_POINTS.times = GAME_POINTS.times + ((4 - GAME_POINTS.game_quarter)*720)
        try:
            # if error_points > 0:
            #     print(f'{error_points} errors with {GAME_POINTS.gameID.iloc[0]}')
            assert GAME_POINTS.times.is_monotonic_decreasing, f'timing is off somewhere for game: {GAME_POINTS.gameID.iloc[0]}'
            home_team_win = GAME_POINTS[['away_team_score', 'home_team_score']].max().values.argmax() if GAME_POINTS['home_team_score'].max() != GAME_POINTS['away_team_score'].max() else -2
            GAME_POINTS['home_team_win'] = home_team_win
            games.append(GAME_POINTS)
        except AssertionError as e:
            print(e)

        PROCESSED_GAMES = pd.concat(games).drop(['point_start_time', 'start_on_offense'], axis=1)
        PROCESSED_GAMES['score_diff'] = PROCESSED_GAMES['home_team_score'] - PROCESSED_GAMES['away_team_score']
    return PROCESSED_GAMES[PROCESSED_GAMES.home_team_win >= 0]

base_url = "https://www.backend.audlstats.com/api/v1/"

endpoint = "games?date=2021:"
ids = pd.DataFrame(requests.get(f'{base_url}{endpoint}').json()['data'])['gameID'].values

game_events_proxy = GameEventsProxy()
all_games = []
for id in ids:
    try:
        game_events = GameEvents(id, game_events_proxy)
        game_events.process_game_events()
        game_df = game_events.get_events_df(True, True, True)
        all_games.append(game_df)
    except AssertionError as e:
        print(id)
        print(e)
    except Exception as e:
        print(id)
        raise e
ALL_GAMES = pd.concat(all_games)
ALL_GAMES.to_csv(f'../data/raw/all_games_{date}.csv', index=False)
df = process_games(ALL_GAMES)
df.to_csv(f'../data/processed/all_games_{date}.csv', index=False)


PLAYER DATA

In [12]:
from audl.stats.endpoints.playerstats import PlayerStats
import pandas as pd
year = 2021
dfs = []
while True:
    df = PlayerStats(year, 'total', 'all').fetch_table()
    df['year'] = year
    dfs.append(df)
    year = year+1
    if len(df) == 0:
        break
player_stats_by_year = pd.concat(dfs)
player_stats_by_year.to_csv(f'../data/raw/player_stat_by_year_{date}')

In [85]:
from audl.stats.endpoints.players import Players
from audl.stats.endpoints.teams import Teams

teams = Teams()
teams_df = teams.get_teams()
teams_df = pd.merge(teams_df.drop('division', axis=1).reset_index(drop=True), teams_df.division.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
teams_df.to_csv(f'../data/raw/teams_{date}')

players = Players()
players_df = players.get_players()
players_df = players_df.explode('teams')
players_df = pd.merge(players_df.drop('teams', axis=1).reset_index(drop=True), players_df.teams.apply(pd.Series).reset_index(drop=True), left_index=True, right_index=True)
players_df['firstName'] = players_df['firstName'].apply(lambda x:str(x).strip())
players_df['lastName'] = players_df['lastName'].apply(lambda x:str(x).strip())
players_df.to_csv(f'../data/raw/players_{date}')

In [11]:
from audl.stats.endpoints.playergamestats import PlayerGameStats
import pandas as pd
playergamestats = PlayerGameStats()

In [19]:
gameID = '2023-06-24-DC-BOS'
game_stats_df = playergamestats.get_request_as_df(f'playerGameStats?gameID={gameID}')
game_stats_df = pd.merge(game_stats_df.player.apply(pd.Series), game_stats_df.drop('player', axis=1), left_index=True, right_index=True)
game_stats_df['fullName'] = game_stats_df['firstName'] + ' ' + game_stats_df['lastName']
stats_cols = ['playerID', 'full_name', 'assists', 'goals', 'hockeyAssists', 'completions', 'throwaways', 'stalls', 'yardsReceived', 'yardsThrown', 'hucksCompleted', 'drops',
 'blocks', 'callahans', 'oPointsPlayed', 'dPointsPlayed']

In [ ]:
['playerID', 'full_name', 'assists', 'goals', 'hockeyAssists', 'completions', 'throwaways', 'stalls', 'yardsReceived', 'yardsThrown', 'hucksCompleted', 'drops',
 'blocks', 'callahans', 'oPointsPlayed', 'dPointsPlayed']

In [1]:
from audl.stats.endpoints.gameevents import GameEventsProxy
game_events = GameEventsProxy()
game_events.get_throws_from_id('2023-06-24-DC-BOS')

In [14]:
rows = []
for event in game_events.current_request['homeEvents']:
    if event['type'] in [18, 19, 20, 22, 23]:
        turnover = 1 if event['type'] in [20, 22, 23] else 0
        event['turnover'] = turnover
        if 'receiverX' not in event:
            event['receiverX'], event['receiverY'] = event['turnoverX'], event['turnoverY']
        event['is_home_team'], event['turnoverX'], event['turnoverY'] = True, None, None
        rows.append(event)

for event in game_events.current_request['awayEvents']:
    if event['type'] in [18, 19, 20, 22, 23]:
        turnover = 1 if event['type'] in [20, 22, 23] else 0
        event['turnover'] = turnover
        if 'receiverX' not in event:
            event['receiverX'], event['receiverY'] = event['turnoverX'], event['turnoverY']
        event['is_home_team'], event['turnoverX'], event['turnoverY'] = False, None, None
        rows.append(event)

pd.DataFrame(rows).drop(['turnoverX', 'turnoverY'], axis=1)

In [7]:
import pandas as pd
from audl.stats.endpoints.gameevents import GameEventsProxy
game_events_proxy = GameEventsProxy()


In [2]:
game_events_proxy.get_penalties_from_id('2023-06-24-DC-BOS')

(4, 6)

In [8]:
pulls = game_events_proxy.get_pulls_from_id('2023-06-24-DC-BOS')

In [20]:
pulls[pulls.is_home_team].in_bounds.sum()

17

In [28]:
(pulls[indexer].pullX == 26.66) | (pulls[indexer].pullX == -26.67)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15     True
16     True
17    False
Name: pullX, dtype: bool

In [29]:
pulls = game_events_proxy.get_pulls_from_id('2023-06-24-DC-BOS')
indexer = pulls.is_home_team
home_team_pullers = pd.DataFrame(pulls[indexer].groupby('puller').puller.count())
home_team_pullers.index.name = None
home_team_pullers.columns = ['Pull Count']
home_team_pullers.sort_values('Pull Count', ascending = False)
home_team_pullers.loc['In Bounds'] = f'{pulls[indexer].in_bounds.sum()} ({pulls[indexer].in_bounds.mean()*100:.1f}%)'
rollers = (pulls[indexer].pullX == 26.66) | (pulls[indexer].pullX == -26.67)
home_team_pullers.loc['Roller'] = f'{rollers.sum()} ({rollers.mean()*100:.1f}%)'
home_team_pullers

,Pull Count
bmccann,9
gcolton,1
rlinehan,7
In Bounds,17 (94.4%)
Roller,2 (11.1%)


In [37]:
from audl.stats.endpoints.gamestats import GameStats
from audl.stats.endpoints.gameevents import GameEventsProxy
game_events_proxy = GameEventsProxy()
game_stats = GameStats('2023-06-24-DC-BOS')
home_pen, away_pen = game_events_proxy.get_penalties_from_id('2023-06-24-DC-BOS')
df = game_stats.get_team_stats()

In [ ]:
team_stats = 

In [ ]:
['completionsNumer', 'completionsDenom', 'hucksNumer', 'hucksDenom', 'blocks', 'turnovers', 'redZoneConversionPerc']

In [53]:
def get_frac_string(row, names):
    for name in names:
        numerator = f'{name}Numer'
        denominator = f'{name}Denom'
        row[f'{name}'] = f'{row[numerator]}/{row[denominator]} ({row[numerator]/row[denominator]:.1f}%)'
    return row


In [54]:
df.apply(get_frac_string, args=('completions',) ,axis=1)

KeyError: 'cNumer'

In [39]:
df.T

,0,0
id,2997,2998
teamSeasonId,279,257
gameId,2989,2989
source,sk-v2,sk-v2
startOnOffense,False,True
updateMoment,2023-06-24T23:02:58.925Z,2023-06-27T22:21:30.408Z
statusId,100,100
completionsNumer,262,323
completionsDenom,287,344
hucksNumer,9,2
